# Blockchain Demo

Let's build a simple blockchain in Python!

In [4]:
import abc
import types
import codecs
import hashlib
import msgpack
import binascii
import collections
from typing import Any, Optional, NamedTuple, List, Dict, DefaultDict, Union
from ipywidgets import interact, interactive
from IPython.display import display
from pydantic import BaseModel
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.serialization import PublicFormat, PrivateFormat, Encoding, NoEncryption

`Digest` class for nicer hashes, and `sha256` helper function.

In [5]:
class Digest(bytes):
    @classmethod
    def zero(cls) -> 'Digest':
        return cls(32)

    def __repr__(self) -> str:
        return binascii.hexlify(self).decode('ascii')

def sha256(*vals) -> Digest:
    ctx = hashlib.sha256()
    for val in vals:
        if isinstance(val, str):
            ctx.update(val.encode('utf8'))
        elif isinstance(val, int):
            ctx.update(val.to_bytes(16, 'big'))
        elif hasattr(val, '__digest__'):
            ctx.update(val.__digest__())
        else:
            ctx.update(val)
    return Digest(ctx.digest())

Function which recovers a public key from a signature, given a `check` predicate called to verify the public key.

In [6]:
def recover_pubkey(signature, data, check=None, backend=None, curve=ec.SECP256K1()):
    from cryptography.hazmat.backends.openssl.ec import _EllipticCurvePublicKey
    
    backend = backend or default_backend()

    curve_nid = backend._elliptic_curve_to_nid(curve)

    with backend._tmp_bn_ctx() as ctx:
        ec_cdata = backend._lib.EC_KEY_new_by_curve_name(curve_nid)
        backend.openssl_assert(ec_cdata != backend._ffi.NULL)
        ec_cdata = backend._ffi.gc(ec_cdata, backend._lib.EC_KEY_free)
        
        group = backend._lib.EC_KEY_get0_group(ec_cdata)
        backend.openssl_assert(group != backend._ffi.NULL)

        z_data = sha256(data)
        z_len = (backend._lib.EC_GROUP_get_degree(group) + 7) // 8
        backend.openssl_assert(z_len > 0)
        z_buf = backend._ffi.new("unsigned char[]", z_data[-z_len:])
        z = backend._lib.BN_CTX_get(ctx)
        backend._lib.BN_bin2bn(z_buf, z_len, z)
        # print(f'z:     {backend._bn_to_int(z)}')

        sigbuf = backend._ffi.new("unsigned char[]", signature)
        psigbuf = backend._ffi.new("unsigned char **", sigbuf)
        sig = backend._lib.d2i_ECDSA_SIG(backend._ffi.NULL, psigbuf, len(signature))
        backend.openssl_assert(sig != backend._ffi.NULL)

        pr = backend._ffi.new("BIGNUM **")
        ps = backend._ffi.new("BIGNUM **")
        backend._lib.ECDSA_SIG_get0(sig, pr, ps)
        r = backend._bn_to_int(pr[0])
        s = backend._bn_to_int(ps[0])
        # print(f'sig:   {r}\n       {s}')

        for y in [0, 1]:
            point = backend._lib.EC_POINT_new(group)
            backend._lib.EC_POINT_set_compressed_coordinates_GFp(group, point, pr[0], y, ctx)
            bnx = backend._lib.BN_CTX_get(ctx)
            bny = backend._lib.BN_CTX_get(ctx)
            backend._lib.EC_POINT_get_affine_coordinates_GFp(group, point, bnx, bny, ctx)
            # print(f'point: {backend._bn_to_int(bnx)}\n       {backend._bn_to_int(bny)}')

            order = backend._lib.BN_CTX_get(ctx)
            backend._lib.EC_GROUP_get_order(group, order, ctx)
            # print(f'order: {backend._bn_to_int(order)}')

            inv = backend._lib.BN_CTX_get(ctx)
            backend._lib.BN_mod_inverse(inv, pr[0], order, ctx)
            # print(f'r inv: {backend._bn_to_int(inv)}')

            rs = backend._lib.BN_CTX_get(ctx)
            backend._lib.BN_mod_mul(rs, inv, ps[0], order, ctx)
            # print(f'r1 s:  {backend._bn_to_int(rs)}')

            rz = backend._lib.BN_CTX_get(ctx)
            rzn = backend._lib.BN_CTX_get(ctx)
            zero = backend._lib.BN_CTX_get(ctx)
            backend._lib.BN_mod_mul(rz, inv, z, order, ctx)
            backend._lib.BN_mod_sub(rzn, zero, rz, order, ctx)
            # print(f'r1 z:  {backend._bn_to_int(rz)}')
            # print(f'-r1 z: {backend._bn_to_int(rzn)}')

            zn = backend._lib.BN_CTX_get(ctx)
            backend._lib.BN_mod_sub(zn, zero, z, order, ctx)

            res = backend._lib.EC_POINT_new(group)
            backend._lib.EC_POINT_mul(group, res, rzn, point, rs, ctx)
            bnx = backend._lib.BN_CTX_get(ctx)
            bny = backend._lib.BN_CTX_get(ctx)
            backend._lib.EC_POINT_get_affine_coordinates_GFp(group, res, bnx, bny, ctx)
            # print(f'pkey:  {backend._bn_to_int(bnx)}\n       {backend._bn_to_int(bny)}')

            ec_cdata = backend._ec_key_set_public_key_affine_coordinates(ec_cdata, backend._bn_to_int(bnx), backend._bn_to_int(bny))
            evp_pkey = backend._ec_cdata_to_evp_pkey(ec_cdata)

            pkey = _EllipticCurvePublicKey(backend, ec_cdata, evp_pkey)
            
            if not check or check(pkey):
                return pkey

## Hashes

* Deterministic "one-way" function
* Uniformly distributed over its range
* No correlation to its input

In [7]:
sha256('')

e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855

In [8]:
sha256('hello')

2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824

In [9]:
sha256('jello')

187c9bceeb919e1b3e6d20fa50ecabf7d9d50b5343e8f9a3d912abb13929102e

In [10]:
sha256('monkey')

000c285457fc971f862a79b786476c78812c8897063c6fa9c045f579a3b2d63f

## Public Key Cryptography

* Powers technologies like TLS, SSL, and PGP
* Applications:
  - Encryption: encrypt with public key, decrypt with associated private key
  - Signing: sign with private key, verify signature with associated public key

In [11]:
private_key = ec.generate_private_key(ec.SECP256K1(), backend=default_backend())
public_key = private_key.public_key()

In [12]:
data = b'my message!'
signature = private_key.sign(data, ec.ECDSA(hashes.SHA256()))
print(codecs.encode(signature, 'hex'))

b'304402203617a402389defb35ab864f5837d71af7a6af41cd2d4dddbc3646d1a346270520220479110cceb32befbd878d36e433d8b5f7eb04a332b3fc0bd40572e456d0f5ca7'


In [13]:
public_key.verify(signature, data, ec.ECDSA(hashes.SHA256()))

## Merkle Trees

In [14]:
class MerkleTree(List[Any]):
    @property
    def hash(self) -> Digest:
        if not self:
            return Digest.zero()

        layer = [sha256(b'\x00', item) for item in self]

        while len(layer) > 1:
            layer = [sha256(b'\x01', *layer[i:i + 2]) for i in range(0, len(layer), 2)]

        return layer[0]

## Addresses

* Hash of a public key
* Funds can only be spent by proving ownership of private key

In [15]:
class Address(Digest):
    private_key: Optional[ec.EllipticCurvePrivateKey] = None
    public_key: Optional[ec.EllipticCurvePublicKey] = None

    def __new__(cls, private_key=None, public_key=None):
        if not public_key and not private_key:
            # generate keypair
            private_key = ec.generate_private_key(ec.SECP256K1(), backend=default_backend())
            public_key = private_key.public_key()
        elif private_key:
            public_key = private_key.public_key()

        # address is SHA-256 of public key
        result = super().__new__(cls, sha256(public_key.public_numbers().encode_point()))
        result.private_key = private_key
        result.public_key = public_key
        return result

In [16]:
Address()

4c4be0f5e8a3e82579028dc2c8012b33346b95ec84a5b949307f67e96d1becfc

## Transactions

* Consists of sender + recipient addresses and amount to transfer
* Nonce is used as a counter of transactions from an account
* Signed using sender's private key

In [17]:
class Transaction(BaseModel):
    sender: Digest
    recipient: Digest
    nonce: int
    amount: int
    signature: Optional[bytes] = None

    def sign(self, key: ec.EllipticCurvePrivateKey):
        data = msgpack.packb(self.values(exclude={'signature'}))

        self.signature = key.sign(data, ec.ECDSA(hashes.SHA256()))

    def verify(self):
        if not self.signature:
            raise Exception('no signature')

        data = msgpack.packb(self.values(exclude={'signature'}))

        # recover public key from signature, verify that it matches sender
        public_key = recover_pubkey(
            self.signature,
            data,
            check=lambda key: Address(public_key=key) == self.sender
        )

        if not public_key:
            raise Exception('invalid signature')
        
        public_key.verify(self.signature, data, ec.ECDSA(hashes.SHA256()))

    def __digest__(self):
        return msgpack.packb(self.values())

## State

* Keeps track of all account balances
* Maintained by each node in the network
* Transition function: $State \times Transaction \to State$

Each account has a balance and a nonce (transaction counter):

In [18]:
class Account(BaseModel):
    balance: int = 0
    nonce: int = 0

In [19]:
class State(DefaultDict[Digest, Account]):
    def __init__(self, default_factory=Account, *args, **kwargs):
        super().__init__(default_factory, *args, **kwargs)

    def apply(self, transaction: Transaction) -> 'State':
        sender = self[transaction.sender]
        recipient = self[transaction.recipient]
        
        # check signature
        try:
            transaction.verify()
        except:
            raise Exception('invalid transaction signature')
        
        # check nonce
        if sender.nonce != transaction.nonce:
            raise Exception('invalid transaction nonce')
        
        # check balance
        if sender.balance < transaction.amount:
            raise Exception('insufficient funds')

        newstate = self.copy()
        sender = newstate[transaction.sender] = sender.copy()
        recipient = newstate[transaction.recipient] = recipient.copy()
        
        # increase nonce, transfer funds
        sender.nonce += 1
        sender.balance -= transaction.amount
        recipient.balance += transaction.amount
        
        return newstate

## Blocks

So far, we have a simple ledger that allows secure transfer of funds between accounts. If this is maintained on a single trusted node, then this is basically all that we need. However, if we want to allow a decentralized network of (possibly untrustworthy) nodes to maintain this ledger, then we need a consensus algorithm. Under certain guarantees, leader election via Paxos/Raft can be used. Proof-of-work (aka Nakamoto) consensus has a stronger guarantee of Byzantine fault tolerance (BFT) given a majority of honest nodes.

In [20]:
class Block(BaseModel):
    prev_hash: Digest = Digest.zero()
    nonce: int = 0
    beneficiary: Digest = Digest.zero()
    transactions: MerkleTree

    def __init__(self, *args, **kwargs):
        kwargs.setdefault('transactions', MerkleTree())
        super().__init__(*args, **kwargs)

    def __json__(self):
        return dict(self.values(), hash=self.hash)
        
    @property
    def hash(self) -> Digest:
        return sha256(self.prev_hash, self.nonce, self.beneficiary, self.transactions.hash)

    @property
    def is_valid(self) -> bool:
        return self.hash.startswith(b'\x00')
    
    def mine(self):
        while not self.is_valid:
            self.nonce += 1

In [21]:
interact(lambda nonce=0: Block(nonce=nonce).__json__(), nonce=(0, 1024, 1));

In [22]:
class Blockchain(BaseModel):
    blocks: Dict[Digest, Block]
    best_block: Block = None
    next_block: Block = None
    state: State

    @property
    def best_block_hash(self):
        return self.best_block.hash if self.best_block else Digest.zero()
    
    def mine_block(self, beneficiary=None):
        block = self.next_block
        
        if not block:
            block = self.next_block = Block(prev_hash=self.best_block_hash)
        
        if beneficiary:
            block.beneficiary = beneficiary
        
        block.mine()
        
        self.blocks[block.hash] = self.best_block = block
        self.state[block.beneficiary].balance += 100
        self.next_block = None
        
    def add_transaction(self, transaction):
        if not self.next_block:
            self.next_block = Block(prev_hash=self.best_block_hash)
            
        self.state = self.state.apply(transaction)
        self.next_block.transactions.append(transaction)

In [23]:
alice = Address(); print(f'alice: {alice!r}')
bob = Address(); print(f'bob:   {bob!r}')
carol = Address(); print(f'carol: {carol!r}')

chain = Blockchain(blocks={}, state=State()); chain

alice: f6c8a50cdb8113ed5cc63011d829632a9cef5f20007febb121014228fa553130
bob:   ca71dc47419494fcfb88f33ef676c813a103d86bf48f3a4549edf528ec10830a
carol: acd71df69a0b5dc1cc36cbccfd7754fa54d1ecdeb510eabf5e5d502f015280c7


<Blockchain blocks={} state=defaultdict(<class '__main__.Account'>, {}) best_block=None next_block=None>

In [24]:
chain.mine_block(beneficiary=alice); display(chain.best_block.__json__()); chain.state

{'beneficiary': f6c8a50cdb8113ed5cc63011d829632a9cef5f20007febb121014228fa553130,
 'hash': 001a71a98946769b6898472c381f41c58a18a5dcd89d76248e704f257fce6e50,
 'nonce': 64,
 'prev_hash': 0000000000000000000000000000000000000000000000000000000000000000,
 'transactions': []}

State(__main__.Account,
      {f6c8a50cdb8113ed5cc63011d829632a9cef5f20007febb121014228fa553130: <Account balance=100 nonce=0>})

In [25]:
transaction = Transaction(sender=alice, recipient=bob, nonce=0, amount=25)
transaction.sign(alice.private_key)
chain.add_transaction(transaction)
chain.state

State(__main__.Account,
      {ca71dc47419494fcfb88f33ef676c813a103d86bf48f3a4549edf528ec10830a: <Account balance=25 nonce=0>,
       f6c8a50cdb8113ed5cc63011d829632a9cef5f20007febb121014228fa553130: <Account balance=75 nonce=1>})

In [26]:
chain.mine_block(beneficiary=carol); display(chain.best_block.__json__()); chain.state

{'beneficiary': acd71df69a0b5dc1cc36cbccfd7754fa54d1ecdeb510eabf5e5d502f015280c7,
 'hash': 002f437fbb351e18bfe302be605b4e1e66cae33e7d519fe86164098551d557fa,
 'nonce': 63,
 'prev_hash': 001a71a98946769b6898472c381f41c58a18a5dcd89d76248e704f257fce6e50,
 'transactions': [{'amount': 25,
   'nonce': 0,
   'recipient': ca71dc47419494fcfb88f33ef676c813a103d86bf48f3a4549edf528ec10830a,
   'sender': f6c8a50cdb8113ed5cc63011d829632a9cef5f20007febb121014228fa553130,
   'signature': b'0F\x02!\x00\xf6\x9a+\xb1\x8ao\x97>\xf1\x87*A\xe8\xf5U\xbfO\x8f [X"\xd7\x9dH\xfd$\xa3\x99Q\xad\xad\x02!\x00\xb3ks\x86\xf7\x86\xfc\xd9\x8ctW\x81\x0e\x84U}\xbc+9\r\xd2O\xeaJ\x9b.\xe0\xe8c|\xbbF'}]}

State(__main__.Account,
      {acd71df69a0b5dc1cc36cbccfd7754fa54d1ecdeb510eabf5e5d502f015280c7: <Account balance=100 nonce=0>,
       ca71dc47419494fcfb88f33ef676c813a103d86bf48f3a4549edf528ec10830a: <Account balance=25 nonce=0>,
       f6c8a50cdb8113ed5cc63011d829632a9cef5f20007febb121014228fa553130: <Account balance=75 nonce=1>})